In [1]:
import shapefile  #the pyshp module : Should install pyshp module.
import pandas as pd
from pyproj import Proj, transform  #Should install pyproj module.

In [58]:
# read data (Copy all files from nodelink into nodelink folder: I made it.)
# using old_data
shp_path_node = 'data/[2023-10-13]NODELINKDATA/MOCT_NODE.shp'
sf_node = shapefile.Reader(shp_path_node, encoding = "CP949")
shp_path_link = 'data/[2023-10-13]NODELINKDATA/MOCT_LINK.shp'
sf_link = shapefile.Reader(shp_path_link, encoding = "CP949")

# construct pandas dataframe
#grab the shapefile's field names
# node
fields_node = [x[0] for x in sf_node.fields][1:]
records_node = sf_node.records()
shps = [s.points for s in sf_node.shapes()] # node has coordinate data.
# link
fields_link = [x[0] for x in sf_link.fields][1:]
records_link = sf_link.records()

#write the records into a dataframe
#node
node_dataframe = pd.DataFrame(columns=fields_node, data=records_node)
#add the coordinate data to a column called "coords"
node_dataframe = node_dataframe.assign(coords=shps)
# link
link_dataframe = pd.DataFrame(columns=fields_link, data=records_link)

In [59]:
link_dataframe

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,ROAD_NO,ROAD_NAME,ROAD_USE,MULTI_LINK,...,MAX_SPD,REST_VEH,REST_W,REST_H,C-ITS,LENGTH,UPDATEDATE,REMARK,HIST_TYPE,HISTREMARK
0,2660315900,2660132500,2660132800,1,107,000,-,삼포해변길,0,0,...,30,0,0,0,0,715.687424,20230519,,,
1,2660336100,2660139100,2660139200,1,107,000,-,화암사길,0,0,...,50,0,0,0,0,942.762880,20230519,,,
2,3260017204,3260008202,3260008203,2,103,000,2,남승룡로,0,0,...,60,0,0,0,0,76.039806,20230519,,,
3,3240003900,3240002000,3240002100,3,107,000,-,영산로,0,0,...,60,0,0,0,0,120.458001,20230519,,,
4,3260023002,3260082400,3260089600,3,103,000,17,백강로,0,0,...,50,0,0,0,0,87.589918,20230519,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545707,1610126800,1610048100,1610046800,4,104,000,-,영종대로,0,0,...,50,0,0,0,0,706.106333,20230519,,,
545708,3190270000,3190005100,3190082600,1,107,000,-,종산로,0,0,...,50,0,0,0,0,323.012917,20230519,,,
545709,3960233200,3960089500,3960012801,1,107,000,-,두항길,0,0,...,50,0,0,0,0,1215.130137,20230519,,,
545710,2140742600,2140292900,2140003300,1,107,000,-,승학동길,0,0,...,50,0,0,0,0,69.740102,20230519,,,


In [60]:
type(node_dataframe.iloc[0]['NODE_ID'])

str

In [63]:
# Data restriction
# STNL_REG for Jeju == 405, 406
# NODE_ID for Jeju is between 4050000000 and 4070000000
jeju_node = node_dataframe[node_dataframe['NODE_ID'].str.startswith(('405', '406'))]
jeju_link = link_dataframe[link_dataframe['LINK_ID'].str.startswith(('405', '406'))]
jeju_link

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,ROAD_NO,ROAD_NAME,ROAD_USE,MULTI_LINK,...,MAX_SPD,REST_VEH,REST_W,REST_H,C-ITS,LENGTH,UPDATEDATE,REMARK,HIST_TYPE,HISTREMARK
92,4060004300,4060001600,4060002000,2,106,000,1132,일주동로,0,0,...,60,0,0,0,0,119.578782,20230519,,,
108,4050567300,4050207400,4050207600,1,107,000,-,해맞이해안로,0,0,...,50,0,0,0,0,1345.801153,20230519,,,
145,4050554500,4050199500,4050199700,1,107,000,-,다랑쉬북로,0,0,...,50,0,0,0,0,376.124453,20230519,,,
463,4060260400,4060151500,4060083200,1,107,000,-,태위로,0,0,...,50,0,0,0,0,185.657975,20230519,,,
478,4060445100,4060170700,4060170600,1,107,000,-,표선당포로,0,0,...,30,0,0,0,0,167.859146,20230519,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545232,4060526600,4060185700,4060185800,1,107,000,-,일출로,0,0,...,30,0,0,0,0,121.232754,20230519,,,
545291,4050035501,4050003800,4050073600,1,107,000,-,월산남길,0,0,...,10,0,0,0,0,253.233758,20230519,,,
545377,4050040800,4050018400,4050018600,2,107,000,-,애조로,0,0,...,80,0,0,0,0,735.944433,20230519,,,
545378,4060053100,4060027400,4060027600,1,107,000,-,신서로48번길,0,0,...,50,0,0,0,0,144.455378,20230519,,,


In [65]:
# Change coordinate system
# korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
inProj = Proj(init = 'epsg:5186')
outProj= Proj(init = 'epsg:4326')
latitude = []
longitude= []
for idx,row in jeju_node.iterrows():
    x,y  = row.coords[0][0],row.coords[0][1]  
    nx,ny = transform(inProj,outProj,x,y)     
    latitude.append(ny)
    longitude.append(nx)
jeju_node['latitude'] = latitude
jeju_node['longitude']= longitude
del jeju_node['coords'] # delete coords

c:\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Flutter\bin\ipykernel_19920\485787219.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  nx,ny = transform(inProj,outProj,x,y)     # 새로운 좌표계
C:\Fl

In [68]:
# Change column name to draw network in Gephi
jeju_node.rename(columns={'NODE_ID':'Id'},inplace = True)
jeju_link.rename(columns={'F_NODE':'Source','T_NODE':'Target'},inplace = True)
jeju_node.to_csv('data/Jeju_nodes.csv') # node list
jeju_link.to_csv('data/Jeju_links.csv') # edge(=link) list

C:\Flutter\bin\ipykernel_19920\3515774943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeju_node.rename(columns={'NODE_ID':'Id'},inplace = True)
C:\Flutter\bin\ipykernel_19920\3515774943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jeju_link.rename(columns={'F_NODE':'Source','T_NODE':'Target'},inplace = True)


In [ ]:
import pandas as pd
file_path = 'data/Jeju_traffic.csv'

traffic_df = pd.read_csv(file_path)
column_name=['순번', '생성일', '생성시분', '링크ID', '도로관리기관','통행속도','통행시간']

traffic_df= traffic_df.values.tolist()
traffic_df = pd.DataFrame(traffic_df, columns=column_name)
idx = traffic_df.groupby('링크ID')['통행속도'].idxmax()


traffic_df = traffic_df.loc[idx]
traffic_df.to_csv('data/Jeju_traffic.csv', index=False)



In [ ]:
traffic_df = pd.read_csv('data/Jeju_traffic.csv')


merged_df = pd.merge(traffic_df,jeju_link, left_on='링크ID', right_on='LINK_ID', how='inner')


merged_df.drop('LINK_ID', axis=1, inplace=True)





In [ ]:
merged_df['시간'] = (merged_df['LENGTH_x']*0.06) / merged_df['통행속도']

In [ ]:
merged_df.drop('통행시간', axis=1, inplace=True)
merged_df.drop('ROAD_RANK_x', axis=1, inplace=True)
merged_df.drop('ROAD_TYPE_x', axis=1, inplace=True)
merged_df.drop('REMARK_x', axis=1, inplace=True)
merged_df.drop('HIST_TYPE_x', axis=1, inplace=True)
merged_df.drop('HISTREMARK_x', axis=1, inplace=True)

In [ ]:
merged_df.to_csv('data/Jeju_links.csv', index=False, encoding='utf_8')
merged_df